In [171]:
import pandas as pd
import numpy as np
from IPython.display import Image 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [185]:
treino = pd.read_csv('train.csv')
teste = pd.read_csv('test.csv')

In [186]:
teste.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,41,PR,22,F,3,1,1,5,1,...,40.0,420.0,B,A,C,A,A,C,C,A
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,21,MA,26,F,3,1,1,8,1,...,100.0,580.0,E,B,C,B,B,B,F,A
2,b38a03232f43b11c9d0788abaf060f7366053b6d,23,CE,21,M,1,1,2,0,2,...,80.0,320.0,E,E,D,B,B,A,NaN,A
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,15,PA,27,F,3,1,1,8,1,...,NaN,NaN,H,E,G,B,B,A,NaN,A
4,715494628a50142ce8cb17191cfe6d0f3cae0934,41,PR,18,M,1,1,2,0,2,...,0.0,320.0,D,H,H,C,B,A,NaN,A


In [187]:
treino.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,...,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,...,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,...,NaN,A,A,A,A,A,A,A,A,A


In [212]:
#Verificando a correlação das colunas com a coluna target
corr = treino.corr()['NU_NOTA_MT'].sort_values(ascending=False)
corr.head(20)

NU_NOTA_MT                 1.000000
NU_NOTA_CN                 0.584941
NU_NOTA_CH                 0.529594
NU_NOTA_LC                 0.494695
NU_NOTA_REDACAO            0.379376
TP_DEPENDENCIA_ADM_ESC     0.373233
NU_NOTA_COMP3              0.350307
NU_NOTA_COMP5              0.343337
NU_NOTA_COMP4              0.342282
NU_NOTA_COMP2              0.335638
NU_NOTA_COMP1              0.299402
CO_MUNICIPIO_NASCIMENTO    0.113416
CO_UF_NASCIMENTO           0.112236
CO_MUNICIPIO_PROVA         0.109433
CO_MUNICIPIO_RESIDENCIA    0.108850
CO_UF_PROVA                0.108251
CO_UF_RESIDENCIA           0.107493
CO_MUNICIPIO_ESC           0.102517
CO_ESCOLA                  0.102072
CO_UF_ESC                  0.100935
Name: NU_NOTA_MT, dtype: float64

In [213]:
#Removendo coluna TP_DEPENDENCIA_ADM_ESC por não ser interessante e pegando o top 10 da correlção para ser nossas Features
corr.drop('TP_DEPENDENCIA_ADM_ESC',inplace=True)
coluna_features = corr.index[:10]
coluna_features

Index(['NU_NOTA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_REDACAO', 'NU_NOTA_COMP3', 'NU_NOTA_COMP5', 'NU_NOTA_COMP4',
       'NU_NOTA_COMP2', 'NU_NOTA_COMP1'],
      dtype='object')

In [220]:
#Criando um DataFrame com os dados de Treino, pegamosapenas algumas colunas do Dataset para gerar o outro
treino_dados = treino[coluna_features]
treino_dados.head()

,NU_NOTA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_REDACAO,NU_NOTA_COMP3,NU_NOTA_COMP5,NU_NOTA_COMP4,NU_NOTA_COMP2,NU_NOTA_COMP1
0,399.4,436.3,495.4,581.2,520.0,120.0,80.0,80.0,120.0,120.0
1,459.8,474.5,544.1,599.0,580.0,120.0,80.0,120.0,120.0,140.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
treino_dados.shape,teste.shape

((13730, 10), (4576, 47))

In [222]:
#Retirando do Dataset os valores NaN das notas
treino_dados  = treino_dados.replace(0, np.NaN).dropna(how='all').fillna(0)
teste   = teste.replace(0, np.NaN).dropna(how='all').fillna(0)

In [223]:
treino_dados.shape,teste.shape

((10375, 10), (4576, 47))

In [224]:
#Separando nossas Features de treino ('NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO') e Alvo ('NU_NOTA_MT')
coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']
coluna_label = ['NU_NOTA_MT']

In [225]:
#Dividindo o dataset treino_dados em 2, o dataset de treino e teste
X_treino,X_teste,Y_treino,Y_teste = train_test_split(treino_dados[coluna_features].to_numpy()
                                                     ,treino_dados[coluna_label].to_numpy())

In [226]:
#Vendo quantas linhas em cada dataset 
X_treino.shape,X_teste.shape,Y_treino.shape,Y_teste.shape

((7781, 4), (2594, 4), (7781, 1), (2594, 1))

In [227]:
X_treino

array([[383.8, 554.1, 558. , 540. ],
       [505. , 493.7, 450.9, 500. ],
       [471.6, 457.6, 446.6, 400. ],
       ...,
       [435.2, 490.4, 462.5, 420. ],
       [424.6, 467.5, 462.9, 400. ],
       [496.7, 581.6, 572.5, 680. ]])

In [248]:
modelo = GradientBoostingRegressor(loss='ls',  learning_rate=0.1, subsample=0.1 )
modelo.fit(X_treino, Y_treino)

C:\Users\Lucas\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=0.1, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [249]:
#Realizando predição e checando no mean_squared_error
modelo_predicoes = modelo.predict(X_teste)
avaliacao = mean_squared_error(Y_teste, modelo_predicoes)
print(f"Minha avaliação nos dados de teste foi de {avaliacao}")

Minha avaliação nos dados de teste foi de 5575.104347820636


In [250]:
#Atribuindo nosso modelo para prever as notas de MAT no nosso dataset teste
MODELO = modelo
X_desafio = teste[coluna_features].to_numpy()
predicao_desafio = MODELO.predict(X_desafio)

In [251]:
#Montando nosso dataframe de envio do desafio com o número de inscrição
desafio_df = pd.DataFrame(teste.NU_INSCRICAO)

In [252]:
#Atribuindo a predição ao nosso dataframe com o nome de coluna "NU_NOTA_MT"
desafio_df ['NU_NOTA_MT'] = predicao_desafio

In [253]:
desafio_df['NU_NOTA_MT'][desafio_df['NU_NOTA_MT'] < 0] = 0

C:\Users\Lucas\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [254]:
#Determinando o index do nosso Dataframe
desafio_df.set_index("NU_INSCRICAO",inplace=True)

In [255]:
desafio_df

,NU_NOTA_MT
NU_INSCRICAO,
73ff9fcc02f0a99919906c942c2e1a1042cdcf98,430.477301
71a95f9f1b91a82c65ad94abbdf9f54e6066f968,454.562631
b38a03232f43b11c9d0788abaf060f7366053b6d,548.435735
70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.000000
715494628a50142ce8cb17191cfe6d0f3cae0934,564.353578
...,...
dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,434.013724
a75fa8770257e7c9368d059fe53d9ef431f4bdef,459.593710
655fa6306720ff16e825903b5422a46608a77545,645.076062


In [247]:
#Gerando o arquivo a ser enviado pra o teste
desafio_df.to_csv('answer.csv')